[View in Colaboratory](https://colab.research.google.com/github/ZacCranko/robustlearningexperiments/blob/master/robust_mnist.ipynb)

In [139]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tf

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# Copyright 2018 Google LLC.
# SPDX-License-Identifier: Apache-2.0
def power_iterate_conv(
  weights, strides, padding, input_shape, output_shape, num_iter, u=None):
  """Perform power iteration for a convolutional layer."""
  
  with tf.variable_scope(None, default_name='power_iteration'):
    u_var = tf.get_variable(
       'u_conv', [1] + map(int, output_shape[1:]),
       initializer=tf.random_normal_initializer(),
       trainable=False)
    u = u_var
    
    for _ in xrange(num_iter):
      v = tf.nn.conv2d_transpose(
         u, weights, [1] + map(int, input_shape[1:]), strides, padding)
      v /= tf.sqrt(tf.maximum(2 * tf.nn.l2_loss(v), 1e-12))
      u = tf.nn.conv2d(v, weights, strides, padding)
      u /= tf.sqrt(tf.maximum(2 * tf.nn.l2_loss(u), 1e-12))
      
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, tf.assign(u_var, u))
    
    return tf.reduce_sum(u * tf.nn.conv2d(v, weights, strides, padding))
  
def power_iterate(weights, num_iter, u=None):
  """Perform power iteration for a fully connected layer."""
  output_shape, input_shape = weights.get_shape().as_list()

  with tf.variable_scope(None, default_name='power_iteration'):
    u_var = tf.get_variable(
       'u_conv',  map(int, [output_shape]) + [1],
       initializer=tf.random_normal_initializer(),
       trainable=False)
    u = u_var

    for _ in xrange(num_iter):
      v = tf.matmul(weights, u, transpose_a=True)
      v /= tf.sqrt(tf.maximum(2 * tf.nn.l2_loss(v), 1e-12))
      u = tf.matmul(weights, v)
      u /= tf.sqrt(tf.maximum(2 * tf.nn.l2_loss(u), 1e-12))

    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, tf.assign(u_var, u))

    return tf.reduce_sum(u * tf.matmul(weights, v))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob  = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

lip = \
  1/4 * power_iterate_conv(W_conv1, [1,1,1,1], 'SAME', x_image.get_shape(), h_conv1.get_shape(), 5) ** 4 +\
  1/4 * power_iterate_conv(W_conv2, [1,1,1,1], 'SAME', h_pool1.get_shape(), h_conv2.get_shape(), 5) ** 4 +\
  1/4 * power_iterate(W_fc1, 5) ** 4 +\
  1/4 * power_iterate(W_fc2, 5) ** 4

In [145]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)) + lip
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_robust_crossent = cross_entropy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g, training robust crossent %g' % (i, train_accuracy, train_robust_crossent))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.08, training robust crossent 7.31238
step 100, training accuracy 0.88, training robust crossent 0.558542
step 200, training accuracy 0.92, training robust crossent 0.356821
step 300, training accuracy 0.94, training robust crossent 0.187732
step 400, training accuracy 0.98, training robust crossent 0.129108
step 500, training accuracy 0.92, training robust crossent 0.306784
step 600, training accuracy 0.98, training robust crossent 0.235584
step 700, training accuracy 1, training robust crossent 0.076114
step 800, training accuracy 0.92, training robust crossent 0.297581
step 900, training accuracy 1, training robust crossent 0.0558992
step 1000, training accuracy 0.96, training robust crossent 0.219842
step 1100, training accuracy 0.92, training robust crossent 0.274248
step 1200, training accuracy 0.94, training robust crossent 0.239046
step 1300, training accuracy 0.98, training robust crossent 0.0755069
step 1400, training accuracy 0.96, training robust 